In [46]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [47]:
import pandas as pd
import numpy as np

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [48]:
# Read out CSV
df = pd.read_csv('training_observables.csv', header=None)
#df = pd.read_csv('training_observables.csv', sep=',')
df

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,0,0.0,0.493280,119.32780,0.026617,0.019539,0.001285,0.004125,-0.114632,-0.192545,...,0.001793,0.003997,0.008152,0.015164,0.025790,0.040464,0.059261,0.081969,0.108310,0.138916
1,1,0.0,0.499796,140.66150,0.027218,0.015303,0.017385,0.001732,1.389377,0.282950,...,0.000811,0.001750,0.003415,0.006048,0.009945,0.015720,0.024593,0.038424,0.059534,0.090801
2,2,0.0,0.503392,152.76870,0.011552,0.016412,0.004306,0.007357,-0.133998,0.223442,...,0.003183,0.006916,0.013742,0.025196,0.042973,0.068877,0.104854,0.153033,0.215902,0.298440
3,3,0.0,0.488244,123.84110,0.016648,0.007232,0.004221,0.001482,1.066655,0.425053,...,0.000763,0.001691,0.003347,0.005789,0.008923,0.013181,0.020897,0.036646,0.066004,0.117576
4,4,0.0,0.496821,130.78380,0.029870,0.034460,0.016352,0.001592,0.849569,-0.411059,...,0.000919,0.002004,0.003871,0.006358,0.008433,0.007623,0.003397,0.026416,0.078568,0.177591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22570,22570,1.0,0.495708,43.28272,0.031320,0.010866,0.007185,0.000848,-0.414526,-0.690625,...,0.000392,0.000845,0.001672,0.003088,0.005341,0.008532,0.012373,0.016067,0.018169,0.015897
22571,22571,1.0,0.536097,35.77644,0.089789,0.015850,0.006881,0.005601,-0.028845,0.934089,...,0.001865,0.003920,0.007764,0.015250,0.029065,0.051837,0.085413,0.130882,0.189289,0.264222
22572,22572,1.0,0.516989,40.66633,0.056547,0.031709,0.012869,0.007707,0.561080,0.025548,...,0.003741,0.007919,0.014907,0.025046,0.037916,0.052361,0.066622,0.078427,0.085027,0.082829
22573,22573,1.0,0.514398,48.36263,0.059926,0.013307,0.006346,0.004572,0.499208,-1.016149,...,0.002169,0.004623,0.008857,0.015174,0.023454,0.033371,0.044792,0.058167,0.075026,0.098845


In [49]:
# Every Feature name
header_list = {
    0 : 'number',
    1 : 'eos_type',
    2 :	'mean_pt',
    3 :	'dndy',
    4 :	'v2',
    5 :	'v3',
    6 :	'v4',
    7 : 'v5',
    8 :	'psi2',
    9 :	'psi3',
    10 : 'psi4',
    11 :	'psi5',
    12 :'ptspec_bin0',
    13 : 'ptspec_bin',
    14 : 'ptspec_bin2',
    15 :	'ptspec_bin3',
    16 : 'ptspec_bin4',
    17 :	'ptspec_bin5',
    18 :	'ptspec_bin6',
    19 :	'ptspec_bin7',
    20 :	'ptspec_bin8',
    21 : 	'ptspec_bin9',
    22 :	'ptspec_bin10',
    23 :	'ptspec_bin11',
    24 :	'ptspec_bin12',
    25 :	'ptspec_bin13',
    26 :	'ptspec_bin14',
    27 :	'v2_ptbin0',
    28 :	'v2_ptbin1',
    29 :	'v2_ptbin2',
    30 :	'v2_ptbin3',
    31 :	'v2_ptbin4',
    32 :	'v2_ptbin5',
    33 :	'v2_ptbin6',
    34 :	'v2_ptbin7',
    35 :	'v2_ptbin8',
    36 :	'v2_ptbin9',
    37 :	'v2_ptbin10',
    38 :	'v2_ptbin11',
    39 :	'v2_ptbin12',
    40 :	'v2_ptbin13',
    41 :	'v2_ptbin14',
    42 :	'v3_ptbin0',
    43 :	'v3_ptbin1',
    44 :	'v3_ptbin2',
    45 :	'v3_ptbin3',
    46 :	'v3_ptbin4',
    47 :	'v3_ptbin5',
    48 :	'v3_ptbin6',
    49 :	'v3_ptbin7',
    50 : 'v3_ptbin8',
    51 : 'v3_ptbin9',
    52 : 'v3_ptbin10',
    53 : 'v3_ptbin11',
    54 : 'v3_ptbin12',
    55 : 'v3_ptbin13',
    56 : 'v3_ptbin14',
    57 : 'v4_ptbin0',
    58 : 'v4_ptbin1',
    59 : 'v4_ptbin2',
    60 : 'v4_ptbin3',
    61 : 'v4_ptbin4',
    62 : 'v4_ptbin5',
    63 : 'v4_ptbin6',
    64 : 'v4_ptbin7',
    65 : 'v4_ptbin8',
    66 : 'v4_ptbin9',
    67 : 'v4_ptbin10',
    68 : 'v4_ptbin11',
    69 : 'v4_ptbin12',
    70 : 'v4_ptbin13',
    71 : 'v4_ptbin14',
    72 : 'v5_ptbin0',
    73 : 'v5_ptbin1',
    74 : 'v5_ptbin2',
    75 : 'v5_ptbin3',
    76 : 'v5_ptbin4',
    77 : 'v5_ptbin5',
    78 : 'v5_ptbin6',
    79 : 'v5_ptbin7',
    80 : 'v5_ptbin8',
    81 : 'v5_ptbin9',
    82 : 'v5_ptbin10',
    83 : 'v5_ptbin11',
    84 : 'v5_ptbin12',
    85 : 'v5_ptbin13',
    86 : 'v5_ptbin14'
}

In [50]:
df = df.rename(columns=header_list)
df.head(5)

,number,eos_type,mean_pt,dndy,v2,v3,v4,v5,psi2,psi3,...,v5_ptbin5,v5_ptbin6,v5_ptbin7,v5_ptbin8,v5_ptbin9,v5_ptbin10,v5_ptbin11,v5_ptbin12,v5_ptbin13,v5_ptbin14
0,0,0.0,0.493280,119.3278,0.026617,0.019539,0.001285,0.004125,-0.114632,-0.192545,...,0.001793,0.003997,0.008152,0.015164,0.025790,0.040464,0.059261,0.081969,0.108310,0.138916
1,1,0.0,0.499796,140.6615,0.027218,0.015303,0.017385,0.001732,1.389377,0.282950,...,0.000811,0.001750,0.003415,0.006048,0.009945,0.015720,0.024593,0.038424,0.059534,0.090801
2,2,0.0,0.503392,152.7687,0.011552,0.016412,0.004306,0.007357,-0.133998,0.223442,...,0.003183,0.006916,0.013742,0.025196,0.042973,0.068877,0.104854,0.153033,0.215902,0.298440
3,3,0.0,0.488244,123.8411,0.016648,0.007232,0.004221,0.001482,1.066655,0.425053,...,0.000763,0.001691,0.003347,0.005789,0.008923,0.013181,0.020897,0.036646,0.066004,0.117576
4,4,0.0,0.496821,130.7838,0.029870,0.034460,0.016352,0.001592,0.849569,-0.411059,...,0.000919,0.002004,0.003871,0.006358,0.008433,0.007623,0.003397,0.026416,0.078568,0.177591


In [51]:
df = df.drop(['number'], axis = 1)
df

,eos_type,mean_pt,dndy,v2,v3,v4,v5,psi2,psi3,psi4,...,v5_ptbin5,v5_ptbin6,v5_ptbin7,v5_ptbin8,v5_ptbin9,v5_ptbin10,v5_ptbin11,v5_ptbin12,v5_ptbin13,v5_ptbin14
0,0.0,0.493280,119.32780,0.026617,0.019539,0.001285,0.004125,-0.114632,-0.192545,0.604534,...,0.001793,0.003997,0.008152,0.015164,0.025790,0.040464,0.059261,0.081969,0.108310,0.138916
1,0.0,0.499796,140.66150,0.027218,0.015303,0.017385,0.001732,1.389377,0.282950,0.155580,...,0.000811,0.001750,0.003415,0.006048,0.009945,0.015720,0.024593,0.038424,0.059534,0.090801
2,0.0,0.503392,152.76870,0.011552,0.016412,0.004306,0.007357,-0.133998,0.223442,-0.593591,...,0.003183,0.006916,0.013742,0.025196,0.042973,0.068877,0.104854,0.153033,0.215902,0.298440
3,0.0,0.488244,123.84110,0.016648,0.007232,0.004221,0.001482,1.066655,0.425053,-0.017385,...,0.000763,0.001691,0.003347,0.005789,0.008923,0.013181,0.020897,0.036646,0.066004,0.117576
4,0.0,0.496821,130.78380,0.029870,0.034460,0.016352,0.001592,0.849569,-0.411059,0.742171,...,0.000919,0.002004,0.003871,0.006358,0.008433,0.007623,0.003397,0.026416,0.078568,0.177591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22570,1.0,0.495708,43.28272,0.031320,0.010866,0.007185,0.000848,-0.414526,-0.690625,0.623115,...,0.000392,0.000845,0.001672,0.003088,0.005341,0.008532,0.012373,0.016067,0.018169,0.015897
22571,1.0,0.536097,35.77644,0.089789,0.015850,0.006881,0.005601,-0.028845,0.934089,0.126117,...,0.001865,0.003920,0.007764,0.015250,0.029065,0.051837,0.085413,0.130882,0.189289,0.264222
22572,1.0,0.516989,40.66633,0.056547,0.031709,0.012869,0.007707,0.561080,0.025548,0.430158,...,0.003741,0.007919,0.014907,0.025046,0.037916,0.052361,0.066622,0.078427,0.085027,0.082829
22573,1.0,0.514398,48.36263,0.059926,0.013307,0.006346,0.004572,0.499208,-1.016149,0.443735,...,0.002169,0.004623,0.008857,0.015174,0.023454,0.033371,0.044792,0.058167,0.075026,0.098845


In [52]:
df = clean_dataset(df)

In [53]:
# Read out CSV and sets/samples creation

#df = pd.read_csv('synthetic_data.csv', sep=',')
df = df.astype(float)
train,test = train_test_split(df, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [54]:
# Review the information related to the dataframe

df.info

<bound method DataFrame.info of        eos_type   mean_pt       dndy        v2        v3        v4        v5  \
0           0.0  0.493280  119.32780  0.026617  0.019539  0.001285  0.004125   
1           0.0  0.499796  140.66150  0.027218  0.015303  0.017385  0.001732   
2           0.0  0.503392  152.76870  0.011552  0.016412  0.004306  0.007357   
3           0.0  0.488244  123.84110  0.016648  0.007232  0.004221  0.001482   
4           0.0  0.496821  130.78380  0.029870  0.034460  0.016352  0.001592   
...         ...       ...        ...       ...       ...       ...       ...   
22570       1.0  0.495708   43.28272  0.031320  0.010866  0.007185  0.000848   
22571       1.0  0.536097   35.77644  0.089789  0.015850  0.006881  0.005601   
22572       1.0  0.516989   40.66633  0.056547  0.031709  0.012869  0.007707   
22573       1.0  0.514398   48.36263  0.059926  0.013307  0.006346  0.004572   
22574       1.0  0.523754   58.44409  0.051211  0.039774  0.009579  0.002640   

       

In [55]:
# Table of the description of the dataframe related to fixed parameters

df.describe()

,eos_type,mean_pt,dndy,v2,v3,v4,v5,psi2,psi3,psi4,...,v5_ptbin5,v5_ptbin6,v5_ptbin7,v5_ptbin8,v5_ptbin9,v5_ptbin10,v5_ptbin11,v5_ptbin12,v5_ptbin13,v5_ptbin14
count,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,...,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000,22573.000000
mean,0.404909,0.511822,107.499991,0.064737,0.024196,0.012249,0.005511,-0.003248,0.016878,-0.005120,...,0.002571,0.005195,0.009765,0.017114,0.028097,0.043605,0.064677,0.092782,0.131627,0.275086
std,0.490885,0.051300,81.587016,0.031721,0.014125,0.008256,0.004168,0.372459,0.605286,0.386630,...,0.001911,0.003586,0.006371,0.010743,0.017159,0.026010,0.037611,0.052304,0.073179,2.872055
min,0.000000,0.387254,13.899080,0.000505,0.000317,0.000086,0.000022,-1.569997,-1.047155,-0.785385,...,0.000025,0.000051,0.000032,0.000050,0.000205,0.000291,0.000380,0.000378,0.001460,0.000995
25%,0.000000,0.476563,53.076120,0.041704,0.013792,0.006258,0.002536,-0.200354,-0.502925,-0.281094,...,0.001227,0.002597,0.005078,0.009123,0.015179,0.023906,0.036021,0.052620,0.076126,0.108737
50%,0.000000,0.499808,85.304530,0.061818,0.021871,0.010410,0.004433,-0.000867,0.022481,-0.004800,...,0.002098,0.004395,0.008513,0.015155,0.025131,0.039324,0.058995,0.085379,0.121716,0.173801
75%,1.000000,0.552509,127.306600,0.084620,0.032024,0.016069,0.007318,0.197197,0.544685,0.275647,...,0.003375,0.006884,0.013031,0.022948,0.037907,0.059070,0.087640,0.125191,0.177276,0.251671
max,1.000000,0.706444,448.806200,0.236732,0.097754,0.062505,0.034186,1.568587,1.047186,0.785370,...,0.017935,0.031661,0.053479,0.085983,0.130754,0.188419,0.258436,0.345037,0.924904,231.044500


In [56]:
# Separation of labels

x_train = train_set
y_train = train_set[['eos_type']]

x_test = test_set
y_test = test_set[['eos_type']]

In [61]:
# Reduce dimensions using PCA so later you can fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [62]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [63]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

eos_type
0.0         59.375
1.0         40.625
dtype: float64

In [113]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

eos_type
0.0         60.0
1.0         40.0
dtype: float64

In [114]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [115]:
import torch

In [116]:
num_layers = 10
weights_init = 0.01 * torch.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = torch.tensor(0.0, requires_grad=True)

print(weights_init, bias_init)

tensor([[[ 0.0077, -0.0066, -0.0023],
         [-0.0013,  0.0012,  0.0067]],

        [[-0.0073,  0.0061,  0.0025],
         [-0.0037, -0.0018, -0.0028]],

        [[ 0.0150,  0.0149,  0.0105],
         [-0.0035, -0.0017, -0.0068]],

        [[-0.0071,  0.0074, -0.0113],
         [-0.0056,  0.0054, -0.0058]],

        [[ 0.0011,  0.0053,  0.0052],
         [ 0.0137, -0.0067,  0.0035]],

        [[ 0.0134,  0.0026, -0.0096],
         [ 0.0247, -0.0004,  0.0016]],

        [[-0.0118,  0.0077, -0.0052],
         [-0.0073, -0.0047,  0.0114]],

        [[ 0.0083, -0.0025, -0.0120],
         [-0.0093, -0.0061,  0.0216]],

        [[-0.0093,  0.0005,  0.0029],
         [-0.0021, -0.0037,  0.0183]],

        [[ 0.0083,  0.0108, -0.0122],
         [ 0.0027,  0.0040,  0.0010]]], grad_fn=<MulBackward0>) tensor(0., requires_grad=True)


In [118]:
num_layers = 50
weights_init1 = 0.01 * torch.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init1 = torch.tensor(0.0, requires_grad=True)
weights_init = weights_init1.detach().numpy()
bias_init = bias_init1.detach().numpy()

print(weights_init, bias_init)

[[[ 3.29508865e-03 -5.99293038e-03  2.81979726e-03]
  [-3.28434777e-04  1.27052516e-02 -8.32627062e-03]]

 [[-6.81412453e-03 -1.02192415e-02  1.13299675e-02]
  [ 1.19225215e-02  3.79665196e-03  8.94273166e-03]]

 [[ 8.47155787e-03  1.89309369e-03 -2.07170602e-02]
  [ 1.38055785e-02  3.90350935e-03  9.42928903e-03]]

 [[ 5.59479580e-04  1.03519927e-03  1.28461025e-03]
  [-1.24086840e-02  1.55158965e-02  1.29302847e-03]]

 [[-8.26296303e-03  8.28558858e-03 -1.86245423e-03]
  [-3.53737501e-03  1.27574650e-03 -8.19874112e-04]]

 [[-4.96394094e-03 -1.75202228e-02  6.32164441e-03]
  [ 8.19747278e-04  5.47838071e-03  3.91791156e-03]]

 [[-2.72039953e-03  9.16953571e-03 -1.52892340e-02]
  [-3.84640484e-03  1.21046472e-02 -5.88924950e-03]]

 [[-7.25625490e-04  1.53144787e-03  5.21332864e-03]
  [-2.08226945e-02  2.58026365e-03 -9.61558241e-03]]

 [[ 3.14266817e-03 -5.56108775e-04  1.33281704e-02]
  [-1.78489294e-02 -9.40035842e-03 -1.44474697e-03]]

 [[-1.50173597e-04 -1.75172351e-02  2.05634302

In [119]:
circuit(weights_init, data[0])

tensor(-0.06130477, requires_grad=True)

In [120]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [121]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [122]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [123]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [124]:
Y1 = torch.tensor(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X1 = torch.tensor(data, requires_grad=False)
Y = Y1.detach().numpy()
X = X1.detach().numpy()

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [-0.08910913476308041, 0.3630427451714867], Y =  1
X = [-1.2028277115233912, -0.5707077824877139], Y = -1
X = [0.4812004285706799, -0.8216445451679839], Y =  1
X = [-0.27980950403430993, 0.026166063408495376], Y =  1
X = [-1.214829095053104, -1.165971023712824], Y =  1


In [127]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [128]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(5):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 0.9608861 | Accuracy: 0.6062500 
Iter:     2 | Cost: 0.9608861 | Accuracy: 0.6062500 
Iter:     3 | Cost: 0.9608861 | Accuracy: 0.6062500 
Iter:     4 | Cost: 0.9608861 | Accuracy: 0.6062500 
Iter:     5 | Cost: 0.9608861 | Accuracy: 0.6062500 


In [129]:
Yte1 = torch.tensor(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte1 = torch.tensor(normalize(x_test), requires_grad=False)
Yte = Yte1.detach().numpy()
Xte = Xte1.detach().numpy()

In [130]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.189939918865611, Accuracy: 45.0%


In [131]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,-1.0
2,-1.0,-1.0
3,-1.0,-1.0
4,1.0,-1.0
5,-1.0,1.0
6,-1.0,-1.0
7,-1.0,-1.0
8,-1.0,1.0
9,1.0,-1.0


In [132]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.58      0.54      0.56        26
         1.0       0.25      0.29      0.27        14

    accuracy                           0.45        40
   macro avg       0.42      0.41      0.41        40
weighted avg       0.47      0.45      0.46        40

0.25
0.2857142857142857
0.26666666666666666
0.41208791208791207
